Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-12
2024-12-13


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SH603986 300
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600104 1100
SH600588 1500
SH600233 1000
SZ000999 300
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    # ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    ('GBDT12', 'EXP_BENCH', '2dd2f7cd31e84a65a299f76402d47b44'), # GBDT BENCH_Set1 r0 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[4091245:MainThread](2024-12-13 21:19:28,364) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[4091245:MainThread](2024-12-13 21:19:28,368) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[4091245:MainThread](2024-12-13 21:19:28,369) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-13 09:00:00
{'SH600570': {'amount': 800}, 'SH603986': {'amount': 300}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'SZ000999': {'amount': 300}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SH603986': {'amount': 1783, 'price': 15.636964797973633}, 'SZ000999': {'amount': 3640, 'price': 3.7280220985412598}, 'SZ002129': {'amount': 4184, 'price': 4.6313066482543945}, 'SH600104': {'amount': 12461, 'price': 1.654193639755249}, 'SZ002459': {'amount': 17480, 'price': 1.1490143537521362}, 'SH600233': {'amount': 7140, 'price': 2.019603729248047}, 'SH600588': {'amount': 7188, 'price': 2.528895139694214}, 'SH600570': {'amount': 3727, 'price': 6.463601112365723}, 'SZ000002': {'amount': 14867, 'price': 1.3010629415512085}, 'cash': 316.0, 'now_account_value': 177872.186527133}, 'init_cash': 316.0}
['SH603986', 'SZ000999', 'SZ002129', 'SH600104', 'SZ00

Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[4091245:MainThread](2024-12-13 21:20:16,443) INFO - qlib.timer - [log.py:127] - Time cost: 47.532s | Loading data Done


[4091245:MainThread](2024-12-13 21:20:16,447) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[4091245:MainThread](2024-12-13 21:20:16,452) INFO - qlib.timer - [log.py:127] - Time cost: 0.004s | CSZScoreNorm Done


[4091245:MainThread](2024-12-13 21:20:16,454) INFO - qlib.timer - [log.py:127] - Time cost: 0.009s | fit & process data Done


[4091245:MainThread](2024-12-13 21:20:16,455) INFO - qlib.timer - [log.py:127] - Time cost: 47.545s | Init data Done


[4091245:MainThread](2024-12-13 21:21:03,999) INFO - qlib.timer - [log.py:127] - Time cost: 47.540s | Loading data Done


[4091245:MainThread](2024-12-13 21:21:04,005) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[4091245:MainThread](2024-12-13 21:21:04,021) INFO - qlib.timer - [log.py:127] - Time cost: 0.014s | CSZScoreNorm Done


[4091245:MainThread](2024-12-13 21:21:04,022) INFO - qlib.timer - [log.py:127] - Time cost: 0.021s | fit & process data Done


[4091245:MainThread](2024-12-13 21:21:04,023) INFO - qlib.timer - [log.py:127] - Time cost: 47.565s | Init data Done


In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-12 SZ002841   -0.035522
           SZ002916    0.001608
           SZ002920   -0.050494
           SZ002938   -0.011001
           SZ003816   -0.014815

[2313 rows x 1 columns]

Load and run model

In [8]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        top110ic, top110win = simp_eval(res_df, 110, 10, day=day)
        top120ic, top120win = simp_eval(res_df, 120, 10, day=day)
        top130ic, top130win = simp_eval(res_df, 130, 10, day=day)
        top140ic, top140win = simp_eval(res_df, 140, 10, day=day)
        top150ic, top150win = simp_eval(res_df, 150, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win, top110win, top120win, top130win, top140win, top150win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic, top110ic, top120ic, top130ic, top140ic, top150ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df

# Run

In [9]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    pred_lst.append(pred_df)

[4091245:MainThread](2024-12-13 21:21:04,086) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...


[4091245:MainThread](2024-12-13 21:21:04,168) INFO - qlib.workflow - [recorder.py:341] - Recorder 424db349ccd0423d9fd7fa5dea63c5dd starts running under Experiment 2 ...


[4091245:MainThread](2024-12-13 21:21:06,245) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


[4091245:MainThread](2024-12-13 21:21:06,252) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...


[4091245:MainThread](2024-12-13 21:21:06,268) INFO - qlib.workflow - [recorder.py:341] - Recorder bd309854ea08412aaa62ceae43ce7096 starts running under Experiment 4 ...


...

[4091245:MainThread](2024-12-13 21:21:26,448) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


[4091245:MainThread](2024-12-13 21:21:26,456) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...


[4091245:MainThread](2024-12-13 21:21:26,478) INFO - qlib.workflow - [recorder.py:341] - Recorder d5b353952ec14f7c829ce78f45269969 starts running under Experiment 3 ...


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-13 SH600000   -0.009761
           SH600009    0.027708
           SH600010    0.049938
           SH600011   -0.007178
           SH600015    0.025280


[4091245:MainThread](2024-12-13 21:21:26,749) INFO - qlib.timer - [log.py:127] - Time cost: 0.001s | waiting `async_log` Done


In [10]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_win_dict5[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [11]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [12]:
hold_lst

['SH603986',
 'SZ000999',
 'SZ002129',
 'SH600104',
 'SZ002459',
 'SH600233',
 'SH600588',
 'SH600570',
 'SZ000002']

In [13]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [14]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [15]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,GBDT12,mean,std,score,close
instrument,,,,,,,,,,,
SH600104,-0.111320,-0.214701,-0.066847,-0.097587,-0.082322,-0.169341,-0.084008,-0.118018,0.049990,-0.420742,18.740000
SH600570,-0.061725,-0.157018,-0.077922,-0.044208,-0.104816,-0.106233,-0.025936,-0.082551,0.040940,-0.292912,30.120001
SZ000002,0.003566,-0.117417,-0.039601,-0.023395,-0.054137,-0.060940,-0.004415,-0.042334,0.037860,-0.150590,8.410000
SH600233,0.018351,-0.007292,-0.024660,0.003227,-0.030560,-0.140674,-0.006496,-0.026872,0.048871,-0.100165,14.419999
SZ002129,0.021738,-0.028968,0.038420,-0.026187,-0.079002,-0.090683,-0.021889,-0.026653,0.043826,-0.100106,9.690001
SZ000999,0.020770,0.004097,-0.001009,0.029749,0.020238,0.037824,0.025959,0.019661,0.012775,0.071788,45.239998
SZ002459,-0.007856,-0.012870,0.056796,0.027362,0.012017,0.085523,0.034826,0.027971,0.032506,0.102824,15.450000
SH600588,0.005867,0.033495,0.058713,0.062000,0.073930,0.110385,0.086972,0.061623,0.031804,0.224027,12.120000
SH603986,0.405449,0.364982,0.558871,0.440126,0.425251,0.370273,0.519375,0.440618,0.067959,1.570815,92.979996


In [16]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top10 = report_top.head(110).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top11 = report_top.head(120).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top12 = report_top.head(130).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top13 = report_top.head(140).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top14 = report_top.head(150).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [17]:
sty_report_top0

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,GBDT12,mean,std,score,close
instrument,,,,,,,,,,,
SH603986,0.405449,0.364982,0.558871,0.440126,0.425251,0.370273,0.519375,0.440618,0.067959,1.570815,92.979996
SZ000786,0.065368,0.085758,0.095092,0.105350,0.183358,0.203948,0.130351,0.124175,0.047867,0.448626,29.220001
SH601225,0.067076,0.099024,0.070995,0.083752,0.111996,0.223864,0.089904,0.106659,0.049954,0.385483,24.610001
SH601336,0.062726,0.109859,0.082943,0.114554,0.085940,0.142933,0.115032,0.101998,0.024577,0.366943,49.880001
SH601111,0.078250,0.041640,0.073416,0.103357,0.109893,0.124941,0.099537,0.090148,0.025739,0.325039,8.610001
SH605499,0.017727,0.129702,0.120046,0.022613,0.075594,0.232533,0.016809,0.087861,0.073768,0.315821,251.269989
SH601888,0.051986,0.036523,0.069789,0.066944,0.079816,0.214902,0.053478,0.081920,0.055839,0.297756,71.300003
SH601100,0.042715,0.069073,0.044208,0.081105,0.105083,0.164073,0.061419,0.081097,0.039327,0.294017,54.980000
SH600115,0.090703,0.109351,0.036596,0.107129,0.112084,0.005185,0.109578,0.081518,0.039787,0.289797,4.250000


In [18]:
sty_report_top1

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,GBDT12,mean,std,score,close
instrument,,,,,,,,,,,
SH603486,0.083189,0.142741,0.107709,0.065513,0.059502,0.034898,0.056345,0.078557,0.033644,0.276112,50.970001
SZ000333,0.058930,0.127649,0.082539,0.042225,0.068086,0.112785,0.035788,0.075429,0.032011,0.268137,74.000000
SH600732,0.058088,0.078843,0.088284,0.100100,0.103875,0.002829,0.059762,0.070254,0.032125,0.249297,12.860000
SH600919,0.029042,0.082748,0.043211,0.064293,0.077073,0.109934,0.063053,0.067051,0.024560,0.241791,9.139999
SZ002714,0.027226,0.065195,0.054490,0.042835,0.069207,0.161772,0.035637,0.065195,0.041872,0.236033,40.340000
SH600660,0.029726,0.073253,0.062722,0.051657,0.064025,0.134792,0.036250,0.064632,0.032056,0.232987,57.839996
SZ000895,0.050888,0.088227,0.046217,0.049223,0.050623,0.107281,0.048823,0.063040,0.022582,0.225860,25.530001
SH600588,0.005867,0.033495,0.058713,0.062000,0.073930,0.110385,0.086972,0.061623,0.031804,0.224027,12.120000
SH600027,0.058573,0.082547,0.076578,0.054826,0.053934,0.050171,0.061111,0.062534,0.011349,0.221854,5.539999


In [19]:
sty_report_top2

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,GBDT12,mean,std,score,close
instrument,,,,,,,,,,,
SH603899,0.006005,0.034532,-0.002238,0.102660,0.093288,0.094332,0.079798,0.058339,0.041260,0.214238,32.649998
SH601088,0.049040,0.088302,0.043955,0.036323,0.063268,0.101646,0.034553,0.059584,0.024291,0.213052,42.600002
SH601059,0.044186,0.049315,0.055795,0.053918,0.047350,0.112513,0.050744,0.059117,0.022093,0.212980,15.730000
SH600745,0.032320,0.039755,0.085388,0.049558,0.046592,0.103253,0.049306,0.058025,0.024105,0.208584,36.000000
SZ002415,0.042664,0.032797,0.039511,0.066896,0.057744,0.073592,0.087920,0.057303,0.018618,0.206875,30.809999
SH600176,0.036261,0.079914,0.037079,0.086373,0.063263,0.025225,0.066418,0.056362,0.021875,0.201597,11.700000
SH603259,0.047792,0.119356,0.056139,0.036690,0.072048,0.029155,0.038622,0.057115,0.028600,0.201252,56.500000
SZ002241,0.015416,0.027472,0.055981,0.062926,0.080949,0.046046,0.094316,0.054729,0.025877,0.197532,26.690001
SH600309,0.025975,0.034522,0.052674,0.042092,0.068339,0.120258,0.035618,0.054211,0.029861,0.196305,74.750000


In [20]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top3, './tmp/ensemble_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top4, './tmp/ensemble_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_top5, './tmp/ensemble_top5.png',table_conversion='matplotlib')
dfi.export(sty_report_top6, './tmp/ensemble_top6.png',table_conversion='matplotlib')
dfi.export(sty_report_top7, './tmp/ensemble_top7.png',table_conversion='matplotlib')
dfi.export(sty_report_top8, './tmp/ensemble_top8.png',table_conversion='matplotlib')
dfi.export(sty_report_top9, './tmp/ensemble_top9.png',table_conversion='matplotlib')
dfi.export(sty_report_top10, './tmp/ensemble_top10.png',table_conversion='matplotlib')
dfi.export(sty_report_top11, './tmp/ensemble_top11.png',table_conversion='matplotlib')
dfi.export(sty_report_top12, './tmp/ensemble_top12.png',table_conversion='matplotlib')
dfi.export(sty_report_top13, './tmp/ensemble_top13.png',table_conversion='matplotlib')
dfi.export(sty_report_top14, './tmp/ensemble_top14.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')

In [21]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_ic{k}.png',table_conversion='matplotlib')